In [3]:
# Google Drive API
from google.colab import drive

# Drive bağlantısını oluştur
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Veri klasörlerini belirt
okra_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Raisin/Okralı/Absorbance"
normal_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Raisin/Sağlıklı/Absorbance"

# Verileri depolamak için boş bir DataFrame oluştur
tum_veriler = pd.DataFrame()

# .csv dosyalarını oku
for klasor, label in [(okra_klasoru, 'Okra'), (normal_klasoru, 'Normal')]:
    dosyalar = os.listdir(klasor)

    for dosya in dosyalar:
        dosya_yolu = os.path.join(klasor, dosya)
        veri = pd.read_csv(dosya_yolu)
        #veri = veri.fillna(veri.mean())  # NaN değerleri ortalama ile doldur
        veri['Label'] = label  # Etiket sütunu ekle
        tum_veriler = pd.concat([tum_veriler, veri], ignore_index=True)


# Verileri sınıflandırma için hazırla
X = tum_veriler[['Wavelength (nm)', 'Absorbance (AU)']]
y = tum_veriler['Label']

# Veri normalizasyonu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Verileri eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# K-Nearest Neighbors modelini oluştur ve eğit
knn_model = KNeighborsClassifier()

# Pipeline oluştur
pipeline = Pipeline([('scaler', StandardScaler()), ('knn', knn_model)])

# KNN için GridSearchCV kullanarak hiperparametrelerin ayarlanması
param_grid = {'knn__n_neighbors': [3, 5, 7, 9], 'knn__weights': ['uniform', 'distance']}
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# En iyi modeli seç ve eğitim sonuçlarını değerlendir
best_knn_model = grid_search.best_estimator_
y_pred = best_knn_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"KNN Modeli Doğruluk Oranı: {accuracy:.4f}")

# İkinci eğitim sonuçlarına göre ikinci kez eğitim ve değerlendirme
decision_tree_model_second = DecisionTreeClassifier()
decision_tree_model_second.fit(X_train, y_train)
y_pred_second = decision_tree_model_second.predict(X_test)
accuracy_second = accuracy_score(y_test, y_pred_second)
print(f"Decision Tree Modeli Doğruluk Oranı: {accuracy_second:.4f}")


Fitting 5 folds for each of 8 candidates, totalling 40 fits
